In [1]:
int_list = [3, 4, 5, 6, 5, 4, 3, 2, 1, 3, 4, 4, 3, 2, 4, 6, 2, 1, 32, 4, 55, 3, 2, 1, 4, 6, 78, 8, 5, 6, 0, 9, 8, 6, 3, 5, 6, 7, 7, 8, 8, 4, 2]

In [2]:
import multiprocessing as mp
from tqdm.notebook import tqdm

In [3]:
nprocs = mp.cpu_count()
print("the number of CPU cores", nprocs)

the number of CPU cores 8


In [4]:
#we split the int_list into n-sub-lists, let's say that we use 6 cores
def split(iter, n):
    
    #not sure what this is
    k, m = divmod(len(iter), n)
    
    #no clue
    split_data = [iter[i * k + min(i,m): (i+1) * k + min(i+1, m)] for i in range(n)]
    
    #no clue on what this is
    split_data_order_number = [[i,v] for i,v in enumerate(split_data)]
    
    return split_data_order_number

In [5]:
sub_list = split(int_list, 6)

In [6]:
import pprint 
pprint.pprint(list(sub_list))

[[0, [3, 4, 5, 6, 5, 4, 3, 2]],
 [1, [1, 3, 4, 4, 3, 2, 4]],
 [2, [6, 2, 1, 32, 4, 55, 3]],
 [3, [2, 1, 4, 6, 78, 8, 5]],
 [4, [6, 0, 9, 8, 6, 3, 5]],
 [5, [6, 7, 7, 8, 8, 4, 2]]]


In [7]:
#now we initiate a queue and define our multiprocessing version function func

#the queue will save the output
qout1 = mp.Queue()
qout2 = mp.Queue()

def func(x, q1, q2):
    
    #x is our input 
    index = x[0]
    value = x[1]
    res1 = []
    
    print("Job {}".format(index), "starting \n")
    
    #this for viewing the output
    for i in tqdm(value):
        res1.append(i ** 2)
        
    #then save in the queue
    q1.put(res1)
    q2.put(index)
    
    print("job {}".format(index), "finishing \n")

In [8]:
processes = [mp.Process(target = func, args = (sub, qout1, qout2)) for sub in sub_list]

In [ ]:
for p in processes:
    
    p.daemon = True
    p.start()
    
    